# FISH - Pipeline - A Python notebook to visualize FISH data

```
Author: Luis U. Aguilera
Contact Info: luis.aguilera@colostate.edu

Copyright (c) 2021 Munsky Group 
Colorado State University 
Licensed under BSD 3-Clause License.
```

### Libraries

In [ ]:
# Importing libraries
import sys
import pathlib
import warnings
warnings.filterwarnings("ignore")

### Path to source directories

In [ ]:
# Defining directories
current_dir = pathlib.Path().absolute()
fa_dir = current_dir.parents[0].joinpath('src')

# Importing fish_analyses module
sys.path.append(str(fa_dir))
import fish_analyses as fa

# Path to credentials
desktop_path = pathlib.Path.home()/'Desktop'
path_to_config_file = desktop_path.joinpath('config.yml')


In [ ]:
connect_to_NAS = True

mandatory_substring='nuc_102__cyto_202__psfz_350__psfyx_160'
#data_folder_path = pathlib.Path('smFISH_images/Eric_smFISH_images/20230413/DUSP1_DexTimeConcSweep_300pM_75min_041223' )
#cell_id = 335

data_folder_path = pathlib.Path('smFISH_images/Eric_smFISH_images/20230424/DUSP1_DexTimeConcSweep_1nM_75min_041223')
cell_id = 246

convert_to_standard_format =False


### Paths to folders

In [ ]:
# Substring in folder
# connect_to_NAS = True

# standard_format = True
# if standard_format == True:
#     mandatory_substring='nuc_70__cyto_0__psfz_350__psfyx_160__ts_550_400'
#     data_folder_path = pathlib.Path('smFISH_images/Linda_smFISH_images/Confocal/20220714/MS2-CY5_Cyto543_560_woStim')
#     convert_to_standard_format =False
# else:
#     mandatory_substring='nuc_100__cyto_209__psfz_350__psfyx_120'
#     data_folder_path = pathlib.Path('smFISH_images/Sawyer_smFISH_images/A549/Terminator/20230425_DUSP1_A549_DEX_60min_031123')
#     convert_to_standard_format=True
#     number_color_channels=2
#     number_of_fov=1

### Downloading data

In [ ]:
%%capture
list_images, list_masks, complete_dataframe, number_images, number_color_channels,local_folders,local_data_dir, images_metadata = fa.Utilities().extract_images_masks_dataframe( data_folder_path,
                                                                                                                                mandatory_substring, 
                                                                                                                                path_to_config_file,
                                                                                                                                connect_to_NAS)



In [ ]:
%%capture
if convert_to_standard_format == True:
    local_data_dir,masks_dir, list_files_names, _, list_images= fa.Utilities().convert_to_standard_format(data_folder_path=local_data_dir, 
                                                                                                        path_to_config_file=path_to_config_file,
                                                                                                        number_color_channels=number_color_channels,
                                                                                                        number_of_fov=number_of_fov ,
                                                                                                        download_data_from_NAS = False, )
    images_metadata = None



In [ ]:
# Removing images that were not processed.
selected_images = []
max_image_id = images_metadata['Image_id'].max()+1
for i in range (max_image_id):
    processing_status = images_metadata[images_metadata['Image_id'] == i].Processing.values[0]
    if processing_status == 'successful':
        selected_images.append(list_images[i])
list_images = selected_images
number_images = len(selected_images)
print('number of images: ',number_images)

----
# User parameters
___

In [ ]:
# Selecting cell to visualize
#cell_id = 2
# minimal number of mRNA for consider a cluster a transcription site.
min_ts_size = 10 # Use None as the default value used on your existing dataframe

In [ ]:
# Extracting image with cell and specific dataframe
subsection_image_with_selected_cell, df_spots_subsection_coordinates,_,_,selected_image_id = fa.Utilities().image_cell_selection(cell_id=cell_id, 
                                                                                                         list_images=list_images, 
                                                                                                         dataframe=complete_dataframe)

In [ ]:
#shape of the selected sub-section
print(subsection_image_with_selected_cell.shape)

In [ ]:
#subsection_image_with_selected_cell= subsection_image_with_selected_cell[7:10,:,:,:]

# Plotting composed-color channels

In [ ]:
fa.Plots().plot_selected_cell_colors(image=subsection_image_with_selected_cell,
                                    df= df_spots_subsection_coordinates, 
                                    min_ts_size=min_ts_size, 
                                    show_spots=False,
                                    use_gaussian_filter = True,
                                    show_legend = True,
                                    microns_per_pixel = 0.16,
                                    list_channel_order_to_plot=[1,0,2],#[0,3,1],
                                    image_name=None)

# Plotting detected spots on a single color channel 

In [ ]:
fa.Plots().plot_single_cell(image=subsection_image_with_selected_cell, 
                            df=df_spots_subsection_coordinates, 
                            selected_channel=0, 
                            min_ts_size=min_ts_size,
                            show_spots=True,
                            show_legend = True,
                            image_name=None,
                            microns_per_pixel = 0.16)

## Plotting all color channels

In [ ]:
fa.Plots().plot_single_cell_all_channels(image=subsection_image_with_selected_cell, 
                                        df=df_spots_subsection_coordinates, 
                                        min_ts_size=min_ts_size,
                                        image_name=None,
                                        show_spots = False,
                                        microns_per_pixel=0.16)

# Plotting z-slices for all channels

In [ ]:
fa.Plots().plot_cell_all_z_planes(image=subsection_image_with_selected_cell, image_name=None )

# Plotting a color image for a Field of View

In [ ]:
fa.Plots().plot_complete_fov(list_images=list_images,
                            df=complete_dataframe, 
                            number_of_selected_image = selected_image_id, 
                            use_GaussianFilter=True,
                            microns_per_pixel = 0.16,
                            list_channel_order_to_plot=[1,0,2],
                            show_cell_ids=True)

In [ ]:
raise

## Plotting all cells

In [ ]:
fa.Plots().plot_all_cells(list_images, complete_dataframe, selected_channel=1, spot_type=0,min_ts_size=6,show_spots=True,image_name=None,microns_per_pixel=None,show_legend = True,)

In [ ]:
fa.Plots().plot_all_cells_and_spots(list_images, 
                                  complete_dataframe, 
                                  selected_channel=1, 
                                  spot_type=0,
                                  min_ts_size=6,
                                  image_name=None,
                                  microns_per_pixel=0.16,
                                  show_legend = True)